## Linkedin Pages Scrapping

In [258]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import json
import time
import logging
from collections import OrderedDict
import time
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager
from time import time       # Other imports
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import csv
import re
from time import sleep
from getpass import getpass

In [2]:
def login_details():
    username = input('Enter your email registered in LinkedIn : ')
    password = getpass('Enter your password : ')

    return username, password

def load_more_comments(load_comments_class, driver):
    try:
        load_more_button = driver.find_element_by_class_name(load_comments_class)
        print('<',end='',flush=True)
        while True:
            load_more_button.click()
            sleep(5)
            # 5 second sleep works great for medium-speed net...you can increase if this time seems too less and program finishes before loading all comments....you may decrease till 3 if you have fast internet speed
            try:
                load_more_button = driver.find_element_by_class_name(load_comments_class)
            except:
                print('>')
                print("All comments have been displayed!")
                break
            print('=',end='',flush=True)
    except:
        print("All comments are displaying already!")

def extract_emails(comments):
    emails = []
    for comment in comments:
        email_match = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', comment)
        if email_match:
            emails.append(email_match)
        else:
            emails.append('-')
    return emails

def write_data2csv(names, emails, comments, writer):
    for name,email,comment in zip(names,emails,comments):
        writer.writerow([name,
                        email,
                        comment.encode('utf-8')])
        # utf-8 encoding helps to deal with emojis


In [3]:
post_url = 'https://www.linkedin.com/company/pse---produtos-e-servi-os-de-estat-stica-lda/posts/?feedView=all'

### Check the URL Post

In [4]:
def check_post_url(post_url):
    if post_url:
        return post_url
    else:
        print('You haven\'t entered required post_url in Config.py file!')
        choice = input('Do you want to enter url now? (y/N) : ')
        if choice.lower() == 'y':
            post_url = input('Enter url of post: ')
            return post_url
        elif choice.lower() == 'n':
            exit()
        else:
            print('Invalid choice!')
            exit()

In [5]:
post_url = check_post_url(post_url)


### Load the driver

In [100]:
driver = webdriver.Chrome("C:/Users/GSantos/Documents/GABRIEL RAVI/Web Scrapping/chromedriver_win32_2/chromedriver.exe")

In [7]:
driver.get('https://www.linkedin.com/posts/altice-portugal_alticeportugal-embaixadormeo-activity-6743102030740504577-yrlX/')

In [8]:
content = driver.page_source
soup = BeautifulSoup(content)

### Listagem Empresas

In [262]:
lk = pd.read_excel('Lista_Ravi_Linkedin.xlsx')

lk['posts'] = lk['Link']
lk

,Setor,Categoria,Empresa,Link,posts
0,Tecnologia,Data Sciente,PSE,https://www.linkedin.com/company/pse---produto...,https://www.linkedin.com/company/pse---produto...


### Scroll down

In [263]:
from time import sleep
def scroll_down(driver) -> None:
        """Scrolling down the page and taking the URLs"""
#         global listona
#         global listona2
#         listona = []
#         listona2 = []
#         post_url = []
        last_height = 0
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(5)
#             elements = driver.find_elements_by_xpath('//a[@href]')
                       
#             for elem in elements:
#                 urls = elem.get_attribute('href')
#                 listona.append(urls)    
                
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                break
            last_height = new_height
            
#             listona2 += listona
            
# listona_zord = list(dict.fromkeys(listona2))
# posts_url = [ x for x in listona_zord if "https://www.instagram.com/p/" in x ]

In [264]:
scroll_down(driver)

### Coletando Informações do Post

In [265]:
content = driver.page_source
soup = BeautifulSoup(content)

In [266]:
data_post = []
    try:
        data_post = driver.find_element_by_tag_name('abbr').text
    except:
        data_post = ["NaN"]

IndentationError: unexpected indent (<ipython-input-266-d3669bc84966>, line 2)

[<selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="d2a20cfc-4fa7-41f9-a51e-ba6cd47b455a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="d10193d4-9fde-4696-b60a-df4c824a889d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="0e854f3a-1c2b-4994-ac0d-a6b7c443c8ef")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="22cf3be9-7b3f-4675-a235-6e36dfeab11b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="19cfc34e-e18e-4576-9c55-df28efe28e2d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="e363d7ff-ddf2-4186-9422-c8973db9fbfb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="646b1af300a6c60eacd42ed323b24744", element="01dcc6ff-11bd-40bb-962f-cf

['', '', '', '', '', '', '', '', '', '', '']
11
['799 followers', '799 followers', '799 followers', '799 followers', '799 followers', '799 followers', '799 followers', '10,324,449 followers', '799 followers', '799 followers', '799 followers', '799 followers']
12
['1w •\n1 week ago', '1mo •\n1 month ago', '2mo •\n2 months ago', '2mo •\n2 months ago', '2mo • Edited •\n2 months ago', '3mo • Edited •\n3 months ago', '2mo • Edited •\n2 months ago', '2mo •\n2 months ago', '4mo •\n4 months ago', '11mo • Edited •\n11 months ago', '7mo •\n7 months ago', '1yr •\n1 year ago']
12


In [261]:
data_post = []
data = driver.find_elements_by_xpath('//a/div/span[1]')

for i in data:
    data_post.append(i.text)
print(data_post)
len(data_post)

['new new feed updates notifications', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'IBM', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists', '', 'PSE - Your Data Specialists']


25

In [68]:
len(data_post[1::2])

13

In [79]:
data_post = []
data_posts = driver.find_elements_by_xpath('//a/div/span[1]')
data_posts
for i in data_posts:
    data_post.append(i.text)
    
data_post = data_post[1::2]
data_post = data_post[1:]
len(data_post)

12

In [ ]:
def info_post():
    
    global partilhas
    global texto_post
    global data_post
    global contagem_reacoes
    #global autor_post_id
    global tipo_post_video
    global tipo_post_foto
    global tipo_post_evento
    global tipo_post_link
    

        pagename_post = []
        followers_post = []
        data_post = []

        pagename = driver.find_elements_by_xpath('//a/div/span[1]')
        followers = driver.find_elements_by_xpath('//a/div/span[2]')
        data = driver.find_elements_by_xpath('//a/div/span[3]')

        for i in pagename:
            pagename_post.append(i.text)
        pagename_post = pagename_post[1::2]
        pagename_post = pagename_post[1:]
        print(pagename_post)
        print(len(pagename_post))

        for i in followers:
            followers_post.append(i.text)
        print(followers_post)
        print(len(followers_post))

        for i in data:
            data_post.append(i.text)
        print(data_post)
        print(len(data_post))
        


### First get all informations from one post

In [27]:
#comments = driver.find_elements_by_xpath('//span[@class="ember-view"]')
# this is bad because in case of comments with mentions or tags, it doesnt work
comments = driver.find_elements_by_class_name(comment_class)
comments = [comment.text.strip() for comment in comments]

emails = extract_emails(comments)

names = driver.find_elements_by_class_name(name_class)
names = [name.text.split('\n')[0] for name in names]

headlines = driver.find_elements_by_class_name(headline_class)
headlines = [headline.text.split('\n')[0] for headline in headlines]

# #Likes Post
# likes=soup.find('span', attrs={'class':'v-align-middle social-details-social-counts__reactions-count'})
# likes = [re.sub(r'"|\xa0|\'>|Type":|}|\\|aria-label="| pessoa..?','', x) for x in likes]

#Likes Coments
reactions = driver.find_elements_by_class_name('comments-comment-social-bar__reactions-count')
reactions = [reaction.text.split('\n')[0] for reaction in reactions]
reactions

#TimeStamp
timestamps = driver.find_elements_by_class_name('comments-comment-item__timestamp')
timestamps = [timestamp.text.split('\n')[0] for timestamp in timestamps]
timestamps


#print(comments[:])
#print(names[:10])
#print(emails[:10])

[]

In [28]:
timestamps = driver.find_elements_by_class_name('comments-comment-item__timestamp')
timestamps = [timestamp.text.split('\n')[0] for timestamp in timestamps]
timestamps

[]

In [29]:
# reactions = [reaction.text.split('\n')[0] for reaction in reactions]
# comments-comment-item__social-actions
# likes

In [30]:
emails

[]

In [18]:
likes = []

#### Segundo jeito

In [19]:
likes = []
likes2 = []

for a in soup.findAll('div', attrs={'class':'social-details-social-activity update-v2-social-activity'}):
    name=a.find('span', attrs={'class':'v-align-middle social-details-social-counts__reactions-count'})
    
    likes.append(name.text) 


In [20]:
likes

[]

In [21]:
Linkedin_Teste = pd.DataFrame(np.column_stack([names , emails, comments, headlines, reactions, timestamps]), 
                               columns=["names" , "emails", "comments","Headlines","reactions", "timestamps"])

In [22]:
Linkedin_Teste

,names,emails,comments,Headlines,reactions,timestamps


In [19]:
# write_data2csv(names, emails, comments, writer)

# end = time()       # Finishing Time
# time_spent = end-start # Time taken by script

# print('Linkedin post comments scraped in: %.2f minutes (%d seconds)' % (((time_spent)/60),(time_spent)))

# # driver.quit()

In [ ]:
writer = pd.ExcelWriter('Linkedin_Example_scraping.xlsx',
                        engine='xlsxwriter',options={'strings_to_urls': False}) #.format(staName2)
base.to_excel(writer)
writer.close()


####  Old

In [118]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re as re

In [97]:
#Get the Company's Linkedin URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()


Enter the Company Linkedin URL: https://www.linkedin.com/company/pse---produtos-e-servi-os-de-estat-stica-lda/
Enter your linkedin username: gabrielravi79@gmail.com
Enter your linkedin password: @Gr5010940


In [ ]:
# Login

In [102]:
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = driver.find_element_by_id('username')
elementID.send_keys(username)

elementID = driver.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

In [103]:
#Go to company post webpage
driver.get(page + 'posts/')



### Scroll

In [116]:
import time    # Other imports

#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
### Start with Beautiful Soup

In [254]:
#Check out page source code
company_page = driver.page_source  


#Use Beautiful Soup to get access tags
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

### Criar no scroll a function to click on comments

In [152]:
containers

[<div class="occludable-update ember-view" id="ember94">
 <div class="feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative feed-shared-update-v2--e2e artdeco-card ember-view" data-urn="urn:li:activity:6772092720895836160" id="ember139"><div>
 <h2 class="visually-hidden">
       Feed post
     </h2>
 <div class="display-flex feed-shared-actor display-flex feed-shared-actor--with-control-menu ember-view" id="ember140"><a class="app-aware-link feed-shared-actor__container-link relative display-flex flex-grow-1" data-control-id="G1Q1bIA3G64tIuzhtrSDhw==" data-control-name="actor_container" href="https://www.linkedin.com/company/pse---produtos-e-servi-os-de-estat-stica-lda/?miniCompanyUrn=urn%3Ali%3Afs_miniCompany%3A2235025" target="_self">
 <div class="feed-shared-actor__container absolute"></div>
 <div class="feed-shared-actor__image relative" data-control-id="G1Q1bIA3G64tIuzhtrSDhw==" data-control-name="actor_picture">
 <span class="js-feed-shared-actor__avat

In [255]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
comments = []
video_views = []
media_links = []
media_type = []
coments = []

#Looping through the posts and appending them to the lists
for container in containers:
    
    #Try function to make sure its a post and not a promotion
    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper ember-view"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions social-details-social-counts__item"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__comments social-details-social-counts__item"})
        fixed_comments = container.findAll("p", {"class": "feed-shared-main-content comments-comment-item__main-content feed-shared-main-content--comment t-14 t-black t-normal"})
        
        #Appending date and text to lists
        post_dates.append(posted_date.text.strip())
        post_texts.append(text_box.text.strip())


        #Determining media type and collecting relevant info for each type
        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            video_views.append(view_container[1].text.strip().replace(' Views',''))

        except:
            video_views.append('N/A')

        
        #Appending likes and comments if they exist
        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())                           
        except:                                                           
            post_comments.append(0)
            pass

        try:
            coments.append(fixed_comments[0].text)
        except:                                                           
            coments.append(0)
            pass   
        
        
    except:
        pass

In [256]:
# fixed_comments = container.findAll("p", {"class": "feed-shared-main-content comments-comment-item__main-content feed-shared-main-content--comment t-14 t-black t-normal"})
        
# for container in containers:

#         try:
#             coments.append(fixed_comments[0].text)
#     except:                                                           
#             coments.append(0)
#             pass

In [257]:
coments

['Excelente!',
 0,
 0,
 0,
 0,
 0,
 0,
 'Sempre na vanguarda do conhecimento!!',
 0,
 'Parabéns a todos!',
 0]

In [253]:
for i in containers.find_all("li", {"class": "feed-shared-main-content comments-comment-item__main-content feed-shared-main-content--comment t-14 t-black t-normal"}):
    post_comments.append(new_comments.text)
post_comments

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [130]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]


In [131]:
comment_count

['1comment', '0', '0', '0', '0', '1comment', '0', '0', '0']

In [126]:
data = {
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Coments": coments,
    "Post Comments": comment_count,
    "Video Views": video_views,
    "Media Links": media_links
}

df = pd.DataFrame(data)


In [127]:
df

,Date Posted,Media Type,Post Text,Post Likes,Post Comments,Video Views,Media Links
0,1 week ago,Article,ÚLTIMO MÊS para submissão das candidaturas par...,9,1comment,N/A,https://www.pse.pt/premio-academico-rutis-pse-...
1,1 month ago,Image,A PSE - Your Data Specialists e a RUTIS - Rede...,7,0,N/A,https://media-exp1.licdn.com/dms/image/C4E22AQ...
2,2 months ago,Youtube Video,Na conferência “Dados pessoais e Inteligência ...,5,0,N/A,https://www.facebook.com/jornalexpresso/videos...
3,3 months ago,Article,"A PSE - Your Data Specialists, em parceria com...",7,0,N/A,https://www.facebook.com/jornalexpresso/posts/...
4,2 months ago,"Other: Poll, Shared Post, etc","Nuno Maximiano, Analytics Portfolio and Cognit...",0,0,N/A,None
5,4 months ago,Article,A PSE e a RUTIS efetuaram uma Parceria para de...,12,1comment,N/A,https://www.pse.pt/pse-e-rutis-assinam-protoco...
6,11 months ago,Article,Na Páscoa os portugueses respeitaram o confina...,13,0,N/A,https://expresso.pt/coronavirus/2020-04-15-Cov...
7,7 months ago,Image,Foi atribuída a segunda edição do Prémio PSE “...,10,0,N/A,https://media-exp1.licdn.com/dms/image/C5622AQ...
8,1 year ago,Image,Com vista a contribuir para a contenção da pro...,8,0,N/A,https://media-exp1.licdn.com/dms/image/C4D22AQ...


### Old

In [ ]:
data_posts = []
data_posts = driver.find_elements_by_xpath('//a/div')
data_posts
pagename_post = []
followers_post = []
data_post = []

pagename = driver.find_elements_by_xpath('//a/div/span[1]')
followers = driver.find_elements_by_xpath('//a/div/span[2]')
data = driver.find_elements_by_xpath('//a/div/span[3]')

for i in pagename:
    pagename_post.append(i.text)
pagename_post = pagename_post[1::2]
pagename_post = pagename_post[1:]
print(pagename_post)
print(len(pagename_post))

for i in followers:
    followers_post.append(i.text)
print(followers_post)
print(len(followers_post))

for i in data:
    data_post.append(i.text)
print(data_post)
print(len(data_post))